This notebook downloads the latest user snapshot data from the crvUSD subgraph and processes it to find the losses from soft liquidation in different band ranges

This first portion of code manages the downloading and saving the data as a CSV called user_snapshots.csv

In [8]:
import pluck
import pandas as pd

# crvusd subgraph url
subgraph_url = 'https://api.thegraph.com/subgraphs/name/convex-community/crvusd'

# function to get a portion of user data
def get_user_data(skip_snapshots=0, skip_user_states=0):
  query = f"""
  {{
    snapshots(first: 1000, skip: {skip_snapshots}, where: {{userStateSnapshot: true}}) {{
      basePrice
      oraclePrice
      activeBand
      rate
      userStates (first: 1000, skip: {skip_user_states}) {{
        collateral
        stablecoin
        n
        n1
        n2
        debt
        depositedCollateral
        health
        loss
        lossPct
        timestamp
        user {{
          id
        }}
      }}
      market {{
        id
        collateralName
      }}
    }}
  }}
  """
  frame, = pluck.execute(query, column_names="short", url=subgraph_url)
  return frame

# function to get all user data
def fetch_all_user_data():
    skip_snapshots = 0
    snapshot_df = pd.DataFrame()
    
    while True:
        skip_user_states = 0
        
        while True:
            print(f"skip snapshots: {skip_snapshots}, user states: {skip_user_states}, snapshot_df length: {snapshot_df.shape[0]}")
            data = get_user_data(skip_snapshots, skip_user_states)
            if(data.shape[1] == 18) and not data.isin(snapshot_df).all().all():
                snapshot_df = pd.concat([snapshot_df, data], ignore_index=True)
                skip_user_states += 1000
            elif skip_user_states == 0:
                return snapshot_df
            else:
                break
        skip_snapshots += 1000

In [9]:
# fetch all user data
data = fetch_all_user_data()

# clean, make types correct and rename columns
data = data.dropna()
columns_to_int = ['activeBand', 'n', 'n1', 'n2', 'timestamp']
columns_to_float = [
    'basePrice', 'oraclePrice', 'collateral', 'stablecoin', 'debt',
    'depositedCollateral', 'health', 'loss', 'lossPct', 'rate'
]
data[columns_to_int] = data[columns_to_int].astype(int)
data[columns_to_float] = data[columns_to_float].astype(float)
data = data.rename(columns={'id': 'marketId', 'user.id': 'user'})

# create some columns
data['softLiq'] = data['activeBand'] >= data['n1']
data['collateralUsd'] = data['collateral'] * data['oraclePrice']

# save to csv
data.to_csv("user_snapshots.csv", index=False)

skip snapshots: 0, user states: 0, snapshot_df length: 0
skip snapshots: 0, user states: 1000, snapshot_df length: 82037
skip snapshots: 1000, user states: 0, snapshot_df length: 82037
skip snapshots: 1000, user states: 1000, snapshot_df length: 129563
skip snapshots: 2000, user states: 0, snapshot_df length: 129563
skip snapshots: 2000, user states: 1000, snapshot_df length: 431443
skip snapshots: 3000, user states: 0, snapshot_df length: 431443
skip snapshots: 3000, user states: 1000, snapshot_df length: 775489
skip snapshots: 4000, user states: 0, snapshot_df length: 775489
skip snapshots: 4000, user states: 1000, snapshot_df length: 956056
skip snapshots: 5000, user states: 0, snapshot_df length: 956056
skip snapshots: 5000, user states: 1000, snapshot_df length: 1132347
skip snapshots: 6000, user states: 0, snapshot_df length: 1132347


This portion calculates the losses from the user snapshots.  If you have already downloaded the snapshots, you can just run the notebook from here

In [2]:
import pandas as pd

# if you want the data, run the get_data.ipynb notebook first, this will pull all the latest snapshots.
data = pd.read_csv('df_grouped.csv')

# find the portion of collateral in crvUSD and the collateral token as a percentage
data['collateralPct'] = data['collateralUsd']/(data['collateralUsd']+data['stablecoin'])*100
data['stablecoinPct'] = 100-data['collateralPct']

# currently softLiq column is True even when under softliq, let's create new columns to show the real soft liquidation
data['under_softLiq'] = data['collateralPct'].eq(0)
data['real_softLiq'] = (~data['under_softLiq']) & (data['softLiq'])

# find the loan to value ratio
data['ltv'] = data['debt']/(data['collateralUsd']+data['stablecoin'])*100

#data['n1Price'] = data['basePrice'] * (99/100) ** data['n1']
#data['n2Price'] = data['basePrice'] * (99/100) ** (data['n2']+1)
# sort the data by user, marketId and timestamp
data = data.sort_values(by=['user', 'marketId', 'timestamp']).reset_index(drop=True)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json

def format_data(csv_file, aaveLiqRatio):

    # Read the CSV file
    data = pd.read_csv(csv_file)
    data['collateralTotalValue'] = data['collateralUsd']+data['stablecoin']
    data['collateralPct'] = data['collateralUsd']/(data['collateralTotalValue'])*100
    data['stablecoinPct'] = 100-data['collateralPct']
    time_initial = data.iloc[0]['timestamp']
    data['timeDays'] = (data['timestamp']-time_initial)/86400
    data['ltv'] = data['debt']/(data['collateralTotalValue'])*100
    data['timestampYrsDiff'] = (data['timestamp'] - data['timestamp'].shift(1))/365.25/60/60/24
    data['debtDiff'] = data['rate'] * data['debt'] * data['timestampYrsDiff']
    data['debtAccum'] = data['debtDiff'].cumsum()
    data['interestDebt'] = data['debtAccum']/data['collateralTotalValue']*100
    data['totalLossPct'] = data['lossPct'] + data['interestDebt']
    data['depositedCollateralPct'] = data['depositedCollateral']/data['depositedCollateral'].max()*100
    data['healthFixed'] =  data['health'] + (data['stablecoin'] == 0) * data['collateral'] * (data['oraclePrice']-data['n1Price'])/data['debt']
    data['aaveLiqPrice'] = data['debt']/aaveLiqRatio/data['depositedCollateral']
    
    # Round the values to 2 decimal points
    data = data.fillna(0)

    # Convert the preprocessed data to a dictionary
    chart_data = {
        'timeDays': data['timeDays'].apply(lambda x: f"{x:.2f}").tolist(),
        'oraclePrice': data['oraclePrice'].apply(lambda x: f"{x:.2f}").tolist(),
        'lossPct': data['lossPct'].apply(lambda x: f"{x:.2f}").tolist(),
        'totalLossPct': data['totalLossPct'].apply(lambda x: f"{x:.2f}").tolist(),
        'collateralPct': data['collateralPct'].apply(lambda x: f"{x:.2f}").tolist(),
        'stablecoinPct': data['stablecoinPct'].apply(lambda x: f"{x:.2f}").tolist(),
        'health': (data['healthFixed'] * 100).apply(lambda x: f"{x:.2f}").tolist(),
        'ltv': data['ltv'].apply(lambda x: f"{x:.2f}").tolist(),
        'n1Price': data['n1Price'].apply(lambda x: f"{x:.2f}").tolist(),
        'n2Price': data['n2Price'].apply(lambda x: f"{x:.2f}").tolist(),
        'collateralTotalValue': data['collateralTotalValue'].apply(lambda x: f"{x:.2f}").tolist(),
        'interestDebt': data['interestDebt'].apply(lambda x: f"{x:.2f}").tolist(),
        'stablecoin': data['stablecoin'].apply(lambda x: f"{x:.2f}").tolist(),
        'collateralUsd': data['collateralUsd'].apply(lambda x: f"{x:.2f}").tolist(),
        'debt': data['debt'].apply(lambda x: f"{x:.2f}").tolist(),
        'depositedCollateralPct': data['depositedCollateralPct'].apply(lambda x: f"{x:.2f}").tolist(),
        'rate': (data['rate'] * 100).apply(lambda x: f"{x:.2f}").tolist(),
        'aaveLiqPrice': data['aaveLiqPrice'].apply(lambda x: f"{x:.2f}").tolist()
    }
    
    # Save the preprocessed data as a JSON file
    with open(csv_file.split('.')[0]+'.json', 'w') as json_file:
        json.dump(chart_data, json_file)

    return data



In [4]:
# The soft-liquidation loss statistics are misleading in the current form.  E.g. if a user loses 20% of their collateral
# and then pays back most debt and withdraws 80% collateral the statistics will say the user lost 100% of their collateral

# Let's calculate the loss per day while the user is in soft liquidation.  We will remove time periods where the user
# did an action e.g. deposited or withdrew collateral, paid back debt or borrowed more.

# create a lossPctPerDay column which counts the % a user lost between snapshots standardized to a day
data['lossPctPerDay'] = 0

# count the times a user changes their collateral and debt
data['debtActions'] = 0
data['collateralActions'] = 0

# count the days a user is in soft liquidation
data['softLiqDays'] = 0


# need to iterate through data to get the above data.  This is slow but works.
i = 0
length = len(data)

while i < length:

    # get the current row data
    row = data.iloc[i]
    loan_id = row['user'] + row['marketId'] + str(row['depositedCollateral'])
    collat_value = (row['collateralUsd'] + row['stablecoin']) / row['oraclePrice']
    deposited_collat = row['depositedCollateral']
    debt = row['debt']
    timestamp = row['timestamp']
    
    # it the current loan is the same as the previous loan, ie. same user, marketId and depositedCollateral
    # then we can calculate the lost value and log it if they didn't change their debt
    if i > 0 and prev_loan_id == loan_id:

        # lost value is how much the user lost between snapshots in their collateral e.g., WETH
        lost_value = prev_collat_value - collat_value
        time_days_diff = (timestamp - prev_timestamp) / 86400

        # if the debt changed by more than 2% then we log it as an action
        if prev_debt > debt * 1.02 or prev_debt < debt * 0.98:
            data.at[i, 'debtActions'] += 1
        if prev_deposited_collat != deposited_collat:
            data.at[i, 'collateralActions'] += 1
        # else we log the lost value and the time between snapshots
        elif lost_value > 0:
            lossPctPerDay = lost_value / prev_collat_value / time_days_diff
            data.at[i, 'softLiqDays'] = time_days_diff
            data.at[i, 'lossPctPerDay'] = lossPctPerDay

    # set the previous values to the current values
    prev_collat_value = collat_value
    prev_loan_id = loan_id
    prev_timestamp = timestamp
    prev_debt = debt
    prev_deposited_collat = deposited_collat

    # print progress every 10,000 rows
    if i % 10000 == 0:
        print(f"{i / length * 100}%")

    i += 1

0.0%


/tmp/ipykernel_70862/3499068343.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.33708333333333335' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i, 'softLiqDays'] = time_days_diff
/tmp/ipykernel_70862/3499068343.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.26978543208974e-16' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i, 'lossPctPerDay'] = lossPctPerDay


2.3291292550280076%
4.658258510056015%
6.987387765084023%
9.31651702011203%
11.645646275140038%
13.974775530168046%
16.303904785196053%
18.63303404022406%
20.962163295252072%
23.291292550280076%
25.620421805308087%
27.94955106033609%
30.278680315364102%
32.60780957039211%
34.936938825420114%
37.26606808044812%
39.595197335476136%
41.924326590504144%
44.253455845532145%
46.58258510056015%
48.91171435558817%
51.240843610616174%
53.569972865644175%
55.89910212067218%
58.22823137570019%
60.557360630728205%
62.88648988575621%
65.21561914078421%
67.54474839581222%
69.87387765084023%
72.20300690586824%
74.53213616089624%
76.86126541592425%
79.19039467095227%
81.51952392598028%
83.84865318100829%
86.17778243603628%
88.50691169106429%
90.8360409460923%
93.1651702011203%
95.49429945614833%
97.82342871117633%


In [4]:
# get real soft liquidation subset of data
softLiqData = data.loc[data['real_softLiq']].copy()

# create bins for the number of bands a user chose
bins = [3, 9, 19, 35, 50]
labels = ['4-9', '10-19', '20-35', '36-50']
softLiqData.loc[:, 'n_range'] = pd.cut(softLiqData['n'], bins=bins, labels=labels, right=False)

# group the data by the number of bands a user chose
sl_n_stats_binned = softLiqData.groupby(['n_range']).agg({
    'timestamp': 'count',
    'lossPctPerDay': ['min', 'median', 'mean', 'std', 'max'],
    'softLiqDays': 'sum'
}).reset_index(drop=False)

# rename the columns and save to csv
sl_n_stats_binned.columns = ['n_range', 'entries', 'lossPctDay_min', 'lossPctDay_median', 'lossPctDay_mean', 'lossPctDay_std', 'lossPctDay_max', 'softLiqDays']
sl_n_stats_binned.to_csv('soft_liq_n_stats_binned.csv', index=False)
print(sl_n_stats_binned)

  n_range  entries  lossPctDay_min  lossPctDay_median  lossPctDay_mean  \
0     4-9    39357             0.0           0.001285         0.011455   
1   10-19    17423             0.0           0.000360         0.007471   
2   20-35     1027             0.0           0.000162         0.002533   
3   36-50     1948             0.0           0.000092         0.005558   

   lossPctDay_std  lossPctDay_max  softLiqDays  
0        0.026747        0.389285  5634.910556  
1        0.022235        0.430594  2412.896806  
2        0.006361        0.064132   144.597500  
3        0.013641        0.138341   259.940833  


/tmp/ipykernel_65591/3846588840.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sl_n_stats_binned = softLiqData.groupby(['n_range']).agg({


In [5]:
# get real soft liquidation subset of data
softLiqData = data.loc[data['real_softLiq']].copy()

# create bins for the number of bands a user chose
nBins = [3, 9, 19, 35, 50]
nLabels = ['4-9', '10-19', '20-35', '36-50']
softLiqData.loc[:, 'n_range'] = pd.cut(softLiqData['n'], bins=nBins, labels=nLabels, right=False)

lossBins = [0, 0.00001, 0.00005, 0.0002, 0.001, 0.005, 0.02, 0.1, 0.5,9999]
lossLabels = ['0-0.001%', '0.001-0.005%', '0.005-0.02%', '0.02-0.1%', '0.1-0.5%', '0.5-2%', '2-10%', '10-50%', '50%+']
softLiqData.loc[:, 'loss_range'] = pd.cut(softLiqData['lossPctPerDay'], bins=lossBins, labels=lossLabels, right=False)

# group the data by the number of bands a user chose
sl_n_stats_binned = softLiqData.groupby(['n_range', 'loss_range']).agg({
    'timestamp': 'count',
    'softLiqDays': 'sum'
}).reset_index(drop=False)

# rename the columns and save to csv
sl_n_stats_binned.columns = ['n_range', 'loss_range', 'entries', 'softLiqDays']
sl_n_stats_binned.to_csv('soft_liq_n_stats_binned_with_loss_binned.csv', index=False)
print(sl_n_stats_binned)

   n_range    loss_range  entries  softLiqDays
0      4-9      0-0.001%     8427    32.120417
1      4-9  0.001-0.005%      167    48.821111
2      4-9   0.005-0.02%      537   148.822083
3      4-9     0.02-0.1%     1739   587.548333
4      4-9      0.1-0.5%     3791  1095.539861
5      4-9        0.5-2%     4079  1073.534583
6      4-9         2-10%     2073   455.897083
7      4-9        10-50%      217    41.996806
8      4-9          50%+        0     0.000000
9    10-19      0-0.001%     4040    11.346667
10   10-19  0.001-0.005%      102    35.520833
11   10-19   0.005-0.02%      334   128.883611
12   10-19     0.02-0.1%     1058   371.826389
13   10-19      0.1-0.5%     1685   489.120833
14   10-19        0.5-2%     1151   292.609167
15   10-19         2-10%      517   105.742222
16   10-19        10-50%       56    11.640417
17   10-19          50%+        0     0.000000
18   20-35      0-0.001%      318     2.478611
19   20-35  0.001-0.005%       12     4.633194
20   20-35   

/tmp/ipykernel_70862/1288565374.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sl_n_stats_binned = softLiqData.groupby(['n_range', 'loss_range']).agg({


In [8]:
userActionData = data.copy()

groupedLoans = userActionData.groupby(['user', 'marketId']).agg({
    'debtActions': 'sum',
    'collateralActions': 'sum',
    'timestamp': ['min', 'max'],
    'lossPct': 'max',
    'health': 'min',
    'n': 'max',
    'debt': 'max',
    'collateralUsd': 'max',
    'softLiqDays': 'sum'
}).reset_index(drop=False)

groupedLoans.columns = ['user', 'marketId', 'debtActions', 'collateralActions', 'timestamp_min', 'timestamp_max', 'max_lossPct', 'min_health', 'max_n', 'max_debt', 'max_collateralUsd', 'softLiqDays']
groupedLoans['days'] = (groupedLoans['timestamp_max'] - groupedLoans['timestamp_min']) / 86400
groupedLoans.to_csv('grouped_loans.csv', index=False)
pass